# Análisis de datos utilizando PyRoot


###Cargo los paquetes nesesarias

In [1]:
#from IPython.core.interactiveshell import InteractiveShell
import ROOT
import os
import commands
#InteractiveShell.ast_node_interactivity = "all" # Para tener el modo interactivo completo

In [2]:
ROOT.gROOT.Reset()

###Cargo las Librerías nesesarias

In [3]:
#Esta es la ruta a la librería dinámica de Delphes (debe de ser compilada con la misma versión de root del notebook)
#ROOT.gSystem.AddDynamicPath("/scratch/cms/MadGraph/MG5_aMC_v2_5_2/Delphes/")
ROOT.gSystem.AddDynamicPath("/scratch/cms/Delphes/delphes/")

In [4]:
ROOT.gSystem.Load("libDelphes");

###Construcción del TChain

In [5]:
#Obtengo el número y nombre de carpetas run_#
tmp = commands.getstatusoutput('ls /scratch/cms/Datos/Signal_2j_mu-nu-Run0/Events')
#Parto la cadena de caracteres por '\n', y asigno las partes a un arreglo
Runs = tmp[1].split("\n") 

In [6]:
#Cero el objeto TChain
MainChain=ROOT.TChain("Delphes")

In [7]:
#Lleno el objeto MainChain con las salidas de MadGraph-Pythia_Delphes
for run in range(len(Runs)):
    imput = "../Datos/Signal_2j_mu-nu-Run0/Events/"+Runs[run]+"/tag_1_delphes_events.root"
    MainChain.Add(imput)
    

In [8]:
#Creo el objeto ExRootTreeReader
treeReader = ROOT.ExRootTreeReader(MainChain)
numberOfEntries = treeReader.GetEntries()


In [9]:
#Obtengo las ramas que voy a usar
branchJet = treeReader.UseBranch("Jet")
branchMuon = treeReader.UseBranch("Muon")
branchMissingET = treeReader.UseBranch("MissingET")

In [10]:
treeReader.ReadEntry(0)

True

In [11]:
branchJet.GetEntries()

2

In [12]:
jet = branchJet.At(0)

TClass::TClass:0: RuntimeWarning: no dictionary for class CompBase is available


In [13]:
print jet.PT

76.1234283447


In [12]:
# Loop sobre los eventos
for entry in range(0, numberOfEntries):
    #Se carga la entrada espesífica
    treeReader.ReadEntry(entry)

    # Primer corte, al menos 2 jet
    if branchJet.GetEntries() > 1:
    # Take first jet
    jet = branchJet.At(0)


    # Print jet transverse momentum
    print jet.PT

  # If event contains at least 2 electrons
  if branchElectron.GetEntries() > 1:
    # Take first two electrons
    elec1 = branchElectron.At(0)
    elec2 = branchElectron.At(1)

    # Plot their invariant mass
    histMass.Fill(((elec1.P4()) + (elec2.P4())).M())
